<a href="https://colab.research.google.com/github/sam1gpt/tumor_detection_cuda/blob/main/tumor_detection_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
%config Completer.use_jedi = False

In [3]:
!pip install nvcc4jupyter

In [4]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpss89qsqa".


In [5]:
%%writefile grayscale.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>

using namespace cv;

// Macro for checking CUDA errors
#define CHECK_CUDA(call) do {                                  \
    cudaError_t err = call;                                    \
    if (err != cudaSuccess) {                                  \
        printf("CUDA Error at %s:%d - %s\n",                   \
               __FILE__, __LINE__, cudaGetErrorString(err));   \
        exit(EXIT_FAILURE);                                    \
    }                                                          \
} while(0)

// CUDA Kernel for RGB to Grayscale
__global__ void rgbToGrayKernel(unsigned char *rgb, unsigned char *gray, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int idx = (y * width + x) * 3;
        gray[y * width + x] = (unsigned char)((299 * rgb[idx] + 587 * rgb[idx + 1] + 114 * rgb[idx + 2]) / 1000);
    }
}

int main() {
    // Load Image
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);
    Mat image = imread("img8.jpg", IMREAD_COLOR);
    if (image.empty()) {
        printf("Error: Image not found!\n");
        return -1;
    }

    int width = image.cols;
    int height = image.rows;
    printf("Image loaded: %dx%d\n", width, height);

    // Allocate Memory on Host
    unsigned char *h_rgb = image.data;
    unsigned char *h_gray = (unsigned char*)malloc(width * height * sizeof(unsigned char));
    if (!h_gray) {
        printf("Error: Unable to allocate host memory!\n");
        return -1;
    }

    // Allocate Memory on Device
    unsigned char *d_rgb, *d_gray;
    CHECK_CUDA(cudaMalloc(&d_rgb, width * height * 3 * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_gray, width * height * sizeof(unsigned char)));

    // Copy input image data to device
    CHECK_CUDA(cudaMemcpy(d_rgb, h_rgb, width * height * 3 * sizeof(unsigned char), cudaMemcpyHostToDevice));

    // Define CUDA Grid
    dim3 blockSize(16, 16);
    dim3 gridSize((width + 15) / 16, (height + 15) / 16);

    // Run Kernel
    rgbToGrayKernel<<<gridSize, blockSize>>>(d_rgb, d_gray, width, height);

    // Check for CUDA kernel execution errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Kernel Error: %s\n", cudaGetErrorString(err));
        return -1;
    }
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);
    // Copy back to host
    CHECK_CUDA(cudaMemcpy(h_gray, d_gray, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Check if grayscale image is valid
    int sum = 0;
    for (int i = 0; i < width * height; i++) {
        sum += h_gray[i];
    }
    if (sum == 0) {
        printf("Warning: Grayscale image is entirely black! Possible memory issue.\n");
    } else {
        printf("Grayscale conversion successful.\n");
    }

    // Save the grayscale image
    Mat grayImage(height, width, CV_8UC1);
    memcpy(grayImage.data, h_gray, width * height * sizeof(unsigned char));
    imwrite("gray8.jpg", grayImage);

    // Free Memory
    cudaFree(d_rgb);
    cudaFree(d_gray);
    free(h_gray);

    printf("Processing complete. Grayscale image saved as 'gray8.jpg'.\n");
    printf("Time Taken=%f",elapsedTime);
    return 0;
}


Writing grayscale.cu


In [6]:
!nvcc -arch=sm_75 grayscale.cu -o sample `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [7]:

!./sample

Image loaded: 501x518
Grayscale conversion successful.
Processing complete. Grayscale image saved as 'gray8.jpg'.
Time Taken=124.835838

In [8]:
%%writefile gaussian_blur.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>

using namespace cv;

// CUDA error-checking macro
#define CHECK_CUDA(call) \
{                                \
    cudaError_t err = call;       \
    if (err != cudaSuccess) {    \
        printf("CUDA Error at %s:%d - %s\n", \
               __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE);       \
    }                            \
}

// Gaussian Kernel (3x3)
__constant__ float d_kernel[9] = {
    0.0625f, 0.125f, 0.0625f,
    0.125f,  0.25f,  0.125f,
    0.0625f, 0.125f, 0.0625f
};

// CUDA Kernel for Gaussian Blur
__global__ void gaussianBlurKernel(unsigned char *input, unsigned char *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    if (x > 0 && y > 0 && x < width - 1 && y < height - 1) {
        float sum = 0.0f;

        // Apply 3x3 Gaussian filter
        for (int ky = -1; ky <= 1; ky++) {
            for (int kx = -1; kx <= 1; kx++) {
                int idx = (y + ky) * width + (x + kx);
                int kernelIdx = (ky + 1) * 3 + (kx + 1);
                sum += (float)input[idx] * d_kernel[kernelIdx];
            }
        }

        output[y * width + x] = (unsigned char)sum;
    }
    else {
        // Handle border pixels by copying original value
        output[y * width + x] = input[y * width + x];
    }
}

int main() {
    // Load Grayscale Image
    Mat grayImage = imread("gray8.jpg", IMREAD_GRAYSCALE);
    if (grayImage.empty()) {
        printf("Error: Grayscale image not found!\n");
        return -1;
    }
    cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);
 cudaEventRecord(start, 0);


    int width = grayImage.cols;
    int height = grayImage.rows;

    printf("Applying Gaussian Blur on Image: %dx%d\n", width, height);

    // Clone image to avoid modification issues
    Mat grayClone = grayImage.clone();

    // Allocate Host Memory
    unsigned char *h_gray = grayClone.data;
    unsigned char *h_blurred = (unsigned char*)malloc(width * height * sizeof(unsigned char));
    if (!h_blurred) {
        printf("Error: Unable to allocate host memory!\n");
        return -1;
    }

    // Allocate Device Memory
    unsigned char *d_gray, *d_blurred;
    CHECK_CUDA(cudaMalloc(&d_gray, width * height * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_blurred, width * height * sizeof(unsigned char)));

    // Copy input image to device
    CHECK_CUDA(cudaMemcpy(d_gray, h_gray, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));

    // Define CUDA Grid
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Run Gaussian Blur Kernel
    gaussianBlurKernel<<<gridSize, blockSize>>>(d_gray, d_blurred, width, height);

    // Check for errors
    CHECK_CUDA(cudaPeekAtLastError());
    cudaEventRecord(stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);


    // Copy back the blurred image
    CHECK_CUDA(cudaMemcpy(h_blurred, d_blurred, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Save the blurred image
    Mat blurredImage(height, width, CV_8UC1, h_blurred);
    imwrite("blurred8.jpg", blurredImage);

    // Free Memory
    cudaFree(d_gray);
    cudaFree(d_blurred);
    free(h_blurred);

    printf("Gaussian Blur applied. Blurred image saved as 'blurred8.jpg'.\n");
    printf("Time Taken=%f",elapsedTime);
    return 0;
}



Writing gaussian_blur.cu


In [9]:
!nvcc -arch=sm_75 gaussian_blur.cu -o sample `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [10]:
!./sample

Applying Gaussian Blur on Image: 501x518
Gaussian Blur applied. Blurred image saved as 'blurred8.jpg'.
Time Taken=3.903808

In [11]:
%%writefile edge_detection.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>
#include <math.h>

using namespace cv;

#define CHECK_CUDA(call) \
{                                \
    cudaError_t err = call;       \
    if (err != cudaSuccess) {    \
        printf("CUDA Error at %s:%d - %s\n", \
               __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE);       \
    }                            \
}

__global__ void sobelEdgeDetectionKernel(unsigned char *gray, unsigned char *edges, int width, int height, int threshold) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x > 0 && y > 0 && x < width - 1 && y < height - 1) {
        int Gx = (-1 * gray[(y-1)*width + (x-1)]) + (0 * gray[(y-1)*width + (x)]) + (1 * gray[(y-1)*width + (x+1)]) +
                 (-2 * gray[(y)  *width + (x-1)]) + (0 * gray[(y)  *width + (x)]) + (2 * gray[(y)  *width + (x+1)]) +
                 (-1 * gray[(y+1)*width + (x-1)]) + (0 * gray[(y+1)*width + (x)]) + (1 * gray[(y+1)*width + (x+1)]);

        int Gy = (-1 * gray[(y-1)*width + (x-1)]) + (-2 * gray[(y-1)*width + (x)]) + (-1 * gray[(y-1)*width + (x+1)]) +
                 (0 * gray[(y)  *width + (x-1)]) + (0 * gray[(y)  *width + (x)]) + (0 * gray[(y)  *width + (x+1)]) +
                 (1 * gray[(y+1)*width + (x-1)]) + (2 * gray[(y+1)*width + (x)]) + (1 * gray[(y+1)*width + (x+1)]);

        int magnitude = min(255, (int)sqrtf(Gx * Gx + Gy * Gy));

        edges[y * width + x] = (magnitude > threshold) ? 255 : 0;
    } else {
        edges[y * width + x] = 0;
    }
}

int main() {
    // Load Grayscale Image and Apply Gaussian Blur
    Mat grayImage = imread("blurred8.jpg", IMREAD_GRAYSCALE);
    if (grayImage.empty()) {
        printf("Error: Grayscale image not found!\n");
        return -1;
    }

    GaussianBlur(grayImage, grayImage, Size(5, 5), 1.5);  // Smooth image to remove noise

    int width = grayImage.cols;
    int height = grayImage.rows;
    cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);
 cudaEventRecord(start, 0);


    printf("Processing Image: %dx%d\n", width, height);

    // Allocate Host Memory
    unsigned char *h_gray = grayImage.data;
    unsigned char *h_edges = (unsigned char*)malloc(width * height * sizeof(unsigned char));

    // Allocate Device Memory
    unsigned char *d_gray, *d_edges;
    CHECK_CUDA(cudaMalloc(&d_gray, width * height * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_edges, width * height * sizeof(unsigned char)));

    CHECK_CUDA(cudaMemcpy(d_gray, h_gray, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemset(d_edges, 0, width * height * sizeof(unsigned char)));

    // Define CUDA Grid
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    int threshold = 80;  // Higher threshold to remove weak edges
    sobelEdgeDetectionKernel<<<gridSize, blockSize>>>(d_gray, d_edges, width, height, threshold);

    CHECK_CUDA(cudaPeekAtLastError());
    cudaEventRecord(stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);


    CHECK_CUDA(cudaMemcpy(h_edges, d_edges, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Save Edge-detected Image
    Mat edgeImage(height, width, CV_8UC1, h_edges);
    imwrite("edges_clean.jpg", edgeImage);

    // Free Memory
    cudaFree(d_gray);
    cudaFree(d_edges);
    free(h_edges);

    printf("Edge detection improved. Edge-detected image saved as 'edges_clean.jpg'.\n");
    printf("Time Taken=%f",elapsedTime);
    return 0;
}


Writing edge_detection.cu


In [12]:
!nvcc -arch=sm_75 edge_detection.cu -o sample `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [13]:
!./sample

Processing Image: 501x518
Edge detection improved. Edge-detected image saved as 'edges_clean.jpg'.
Time Taken=0.455712

In [14]:
%%writefile morphological_ops.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>

using namespace cv;

// CUDA error-checking macro
#define CHECK_CUDA(call) \
{                                \
    cudaError_t err = call;       \
    if (err != cudaSuccess) {    \
        printf("CUDA Error at %s:%d - %s\n", \
               __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE);       \
    }                            \
}

// CUDA Kernel for Thresholding
__global__ void thresholdKernel(unsigned char *input, unsigned char *output, int width, int height, unsigned char threshold) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    int idx = y * width + x;
    output[idx] = (input[idx] > threshold) ? 255 : 0;
}

// CUDA Kernel for Dilation with 5x5 structuring element
__global__ void dilationKernel(unsigned char *input, unsigned char *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    unsigned char maxVal = 0;

    // 5x5 structuring element
    for (int ky = -2; ky <= 2; ky++) {
        for (int kx = -2; kx <= 2; kx++) {
            int nx = x + kx;
            int ny = y + ky;
            if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                maxVal = (maxVal > input[ny * width + nx]) ? maxVal : input[ny * width + nx];
            }
        }
    }

    output[y * width + x] = maxVal;
}

// CUDA Kernel for Erosion with 5x5 structuring element
__global__ void erosionKernel(unsigned char *input, unsigned char *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    unsigned char minVal = 255;

    // 5x5 structuring element
    for (int ky = -2; ky <= 2; ky++) {
        for (int kx = -2; kx <= 2; kx++) {
            int nx = x + kx;
            int ny = y + ky;
            if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                minVal = (minVal < input[ny * width + nx]) ? minVal : input[ny * width + nx];
            }
        }
    }

    output[y * width + x] = minVal;
}

int main() {
    // Load Edge-detected Image
    Mat edgeImage = imread("edges_clean.jpg", IMREAD_GRAYSCALE);
    if (edgeImage.empty()) {
        printf("Error: Edge-detected image not found!\n");
        return -1;
    }

    int width = edgeImage.cols;
    int height = edgeImage.rows;
    cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);
 cudaEventRecord(start, 0);

    printf("Processing image: %dx%d\n", width, height);

    // Allocate Host Memory
    unsigned char *h_edges = edgeImage.data;
    unsigned char *h_binary = (unsigned char*)malloc(width * height * sizeof(unsigned char));
    unsigned char *h_dilated = (unsigned char*)malloc(width * height * sizeof(unsigned char));
    unsigned char *h_final = (unsigned char*)malloc(width * height * sizeof(unsigned char));

    if (!h_binary || !h_dilated || !h_final) {
        printf("Error: Unable to allocate host memory!\n");
        return -1;
    }

    // Allocate Device Memory
    unsigned char *d_edges, *d_binary, *d_dilated, *d_final;
    CHECK_CUDA(cudaMalloc(&d_edges, width * height * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_binary, width * height * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_dilated, width * height * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_final, width * height * sizeof(unsigned char)));

    // Copy input image to device
    CHECK_CUDA(cudaMemcpy(d_edges, h_edges, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));

    // Define CUDA Grid
    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Apply thresholding to ensure binary input (adjust threshold value as needed)
    thresholdKernel<<<gridSize, blockSize>>>(d_edges, d_binary, width, height, 50);
    CHECK_CUDA(cudaPeekAtLastError());
    CHECK_CUDA(cudaDeviceSynchronize());

    // Copy thresholded image back to host and save for debugging
    CHECK_CUDA(cudaMemcpy(h_binary, d_binary, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));
    Mat binaryImage(height, width, CV_8UC1, h_binary);
    imwrite("binary_edges8.jpg", binaryImage);

    // Apply Closing (Dilation followed by Erosion) to fill tumor region
    dilationKernel<<<gridSize, blockSize>>>(d_binary, d_dilated, width, height);
    CHECK_CUDA(cudaPeekAtLastError());
    CHECK_CUDA(cudaDeviceSynchronize());

    // Save intermediate dilation result for debugging
    CHECK_CUDA(cudaMemcpy(h_dilated, d_dilated, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));
    Mat dilatedImage(height, width, CV_8UC1, h_dilated);
    imwrite("dilated_edges8.jpg", dilatedImage);

    // Apply Erosion to refine the filled region
    erosionKernel<<<gridSize, blockSize>>>(d_dilated, d_final, width, height);
    CHECK_CUDA(cudaPeekAtLastError());
    cudaEventRecord(stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);


    // Copy back the refined edges
    CHECK_CUDA(cudaMemcpy(h_final, d_final, width * height * sizeof(unsigned char), cudaMemcpyDeviceToHost));

    // Save the final tumor-extracted image
    Mat refinedEdges(height, width, CV_8UC1, h_final);
    imwrite("refined_edges8.jpg", refinedEdges);

    // Free Memory
    cudaFree(d_edges);
    cudaFree(d_binary);
    cudaFree(d_dilated);
    cudaFree(d_final);
    free(h_binary);
    free(h_dilated);
    free(h_final);

    printf("Morphological Closing applied. Image saved as 'refined_edges8.jpg'.\n");
    printf("Debug images saved as 'binary_edges8.jpg' and 'dilated_edges8.jpg'.\n");
    printf("Time Taken=%f",elapsedTime);

    return 0;
}


Writing morphological_ops.cu


In [15]:
!nvcc -arch=sm_75 morphological_ops.cu -o sample `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [16]:
!./sample

Processing image: 501x518
Morphological Closing applied. Image saved as 'refined_edges8.jpg'.
Debug images saved as 'binary_edges8.jpg' and 'dilated_edges8.jpg'.
Time Taken=3.508224

In [17]:
%%writefile tumor_detection.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <opencv2/opencv.hpp>

using namespace cv;
using namespace std;

#define CHECK_CUDA(call) \
{ \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA Error at %s:%d - %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE); \
    } \
}

// CUDA Kernel: Apply Otsu’s Thresholding
__global__ void thresholdOtsu(unsigned char *input, unsigned int *output, int width, int height, int otsuThreshold) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = y * width + x;

    if (x < width && y < height) {
        output[idx] = (input[idx] > otsuThreshold) ? 255 : 0;
    }
}

// CUDA Kernel: Region Growing
__global__ void regionGrowing(unsigned int *input, unsigned int *output, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int idx = y * width + x;

    if (x >= width || y >= height) return;

    if (input[idx] == 255) {
        // Spread to neighboring pixels
        if (x > 0) atomicMax(&output[idx - 1], 255);
        if (x < width - 1) atomicMax(&output[idx + 1], 255);
        if (y > 0) atomicMax(&output[idx - width], 255);
        if (y < height - 1) atomicMax(&output[idx + width], 255);
    }
}

int main() {
    // Load Image
    Mat edgeImage = imread("refined_edges8.jpg", IMREAD_GRAYSCALE);
    if (edgeImage.empty()) {
        printf("Error: Refined edges image not found!\n");
        return -1;
    }
    cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);
 cudaEventRecord(start, 0);


    int width = edgeImage.cols;
    int height = edgeImage.rows;

    // Apply Otsu’s Thresholding
    Mat binaryImage;
    threshold(edgeImage, binaryImage, 0, 255, THRESH_BINARY | THRESH_OTSU);
    int otsuThreshold = threshold(edgeImage, binaryImage, 0, 255, THRESH_BINARY | THRESH_OTSU);

    // Allocate Memory
    unsigned char *h_input = edgeImage.data;
    unsigned int *h_thresholded = (unsigned int*)malloc(width * height * sizeof(unsigned int));
    unsigned int *h_output = (unsigned int*)malloc(width * height * sizeof(unsigned int));

    // Device Memory
    unsigned char *d_input;
    unsigned int *d_thresholded, *d_output;
    CHECK_CUDA(cudaMalloc(&d_input, width * height * sizeof(unsigned char)));
    CHECK_CUDA(cudaMalloc(&d_thresholded, width * height * sizeof(unsigned int)));
    CHECK_CUDA(cudaMalloc(&d_output, width * height * sizeof(unsigned int)));

    CHECK_CUDA(cudaMemcpy(d_input, h_input, width * height * sizeof(unsigned char), cudaMemcpyHostToDevice));

    // CUDA Kernel Launch
    dim3 blockSize(16, 16);
    dim3 gridSize((width + 15) / 16, (height + 15) / 16);

    thresholdOtsu<<<gridSize, blockSize>>>(d_input, d_thresholded, width, height, otsuThreshold);
    CHECK_CUDA(cudaDeviceSynchronize());

    regionGrowing<<<gridSize, blockSize>>>(d_thresholded, d_output, width, height);
    cudaEventRecord(stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);


    CHECK_CUDA(cudaMemcpy(h_output, d_output, width * height * sizeof(unsigned int), cudaMemcpyDeviceToHost));

    // Convert output to 8-bit for saving
    Mat tumorMask(height, width, CV_8UC1);
    for (int i = 0; i < width * height; i++) {
        tumorMask.data[i] = (h_output[i] > 0) ? 255 : 0;
    }

    imwrite("tumor_mask8.jpg", tumorMask);

    // Cleanup
    cudaFree(d_input); cudaFree(d_thresholded); cudaFree(d_output);
    free(h_thresholded); free(h_output);

    printf("Tumor detection complete using Otsu’s Thresholding!\n");
    printf("Time Taken=%f",elapsedTime);
    return 0;
}


Writing tumor_detection.cu


In [18]:
!nvcc -arch=sm_75 tumor_detection.cu -o sample `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [19]:
!./sample

Tumor detection complete using Otsu’s Thresholding!
Time Taken=2.885312

In [20]:
%%writefile tumor_extraction.cu
#include <opencv2/opencv.hpp>
#include <cuda_runtime.h>
#include <iostream>

using namespace cv;
using namespace std;

// CUDA Kernel to Apply Mask (Extract Only Tumor)
__global__ void apply_mask(unsigned char *image, unsigned char *mask, unsigned char *output, int width, int height) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= width * height) return;

    // Ensure the mask is strictly binary
    bool isTumor = (mask[idx] > 128); // Any non-black pixel is treated as tumor

    // If mask is tumor, keep original pixel; otherwise, make it black
    output[idx] = isTumor ? image[idx] : 0;
}

int main() {
    // Load the grayscale MRI and tumor mask images
    Mat brainImage = imread("img8.jpg", IMREAD_GRAYSCALE);
    Mat tumorMask = imread("tumor_mask8.jpg", IMREAD_GRAYSCALE);

    if (brainImage.empty() || tumorMask.empty()) {
        cerr << "Error: Unable to load images!" << endl;
        return -1;
    }

    // Ensure both images have the same dimensions
    if (brainImage.size() != tumorMask.size()) {
        cerr << "Error: Image and mask size mismatch!" << endl;
        return -1;
    }
    cudaEvent_t start, stop;
cudaEventCreate(&start);
cudaEventCreate(&stop);
 cudaEventRecord(start, 0);

    int width = brainImage.cols;
    int height = brainImage.rows;
    int size = width * height * sizeof(unsigned char);

    // Convert mask to binary (ensure strict black & white)
    threshold(tumorMask, tumorMask, 128, 255, THRESH_BINARY);

    // Allocate memory for input and output on the device
    unsigned char *d_brain, *d_mask, *d_output;
    cudaMalloc((void**)&d_brain, size);
    cudaMalloc((void**)&d_mask, size);
    cudaMalloc((void**)&d_output, size);

    // Copy images to the device
    cudaMemcpy(d_brain, brainImage.data, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_mask, tumorMask.data, size, cudaMemcpyHostToDevice);

    // Define CUDA grid and block sizes
    int threadsPerBlock = 256;
    int blocksPerGrid = (width * height + threadsPerBlock - 1) / threadsPerBlock;

    // Launch CUDA kernel to extract the tumor
    apply_mask<<<blocksPerGrid, threadsPerBlock>>>(d_brain, d_mask, d_output, width, height);
    cudaEventRecord(stop, 0);
cudaEventSynchronize(stop);
float elapsedTime;
cudaEventElapsedTime(&elapsedTime, start, stop);


    // Retrieve the extracted tumor image from device
    Mat extractedTumor(height, width, CV_8UC1);
    cudaMemcpy(extractedTumor.data, d_output, size, cudaMemcpyDeviceToHost);

    // Save the extracted tumor image
    imwrite("extracted_tumor.jpeg", extractedTumor);
    cout << "Tumor extracted successfully! Saved as extracted_tumor.jpeg" << endl;
    printf("Time Taken=%f",elapsedTime);

    // Free CUDA memory
    cudaFree(d_brain);
    cudaFree(d_mask);
    cudaFree(d_output);

    return 0;
}


Writing tumor_extraction.cu


In [21]:
!nvcc -arch=sm_75 tumor_extraction.cu -o sample `pkg-config --cflags --libs opencv4`

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::buildMaps" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

/usr/include/opencv4/opencv2/stitching/detail/warpers.hpp(235): warning #611-D: overloaded virtual function "cv::detail::PlaneWarper::warp" is only partially overridden in class "cv::detail::AffineWarper"
  class AffineWarper : public PlaneWarper
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(100): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is only partially overridden in class "cv::detail::FeatherBlender"
  class FeatherBlender : public Blender
        ^

/usr/include/opencv4/opencv2/stitching/detail/blenders.hpp(127): warning #611-D: overloaded virtual function "cv::detail::Blender::prepare" is

In [22]:
!./sample

Tumor extracted successfully! Saved as extracted_tumor.jpeg
Time Taken=2.121760